In [2]:
import os
class Job():
    def __init__(self,DIR="big/",FLATTEN="SNAKE",MODEL="ISING1",**kwargs):
        """Pass all the arguments you want different then default for your rnn to be run
        the default subdirectory is big/ and default rnn is a snake-flattened ising model"""
        self.DIR=DIR
        self.FLATTEN=FLATTEN
        self.MODEL=MODEL
        self.kwargs=kwargs
    def create_file(self,file_name="RNNew.sh"):
        """makes the bash script to run the rnn"""
        self.fn = file_name
        fil = open(self.fn,"w")
        fil.write("#!/bin/bash\n#SBATCH --time=24:00:00\n#SBATCH --partition=bumblebee\n#SBATCH --gres=gpu:1\n")
        fil.write("source activate tensorflow\nmodule load cudnn/7.0-9.0\n")
        fil.write("python RNN_MODELS.py %s %s %s"%(self.DIR,self.FLATTEN,self.MODEL)+ self.__kstring())
        fil.close()
    def __kstring(self):
        """Dictionary arguments to command line"""
        return "".join([" %s=%s"%(a,self.kwargs[a]) for a in self.kwargs])
    def run(self,dependency=None,dtype="afterok"):
        """runs the script, can even set dependency (a different job object) and dtype (default afterok)"""
        if dependency==None:
            a = os.popen("sbatch %s"%self.fn).read()
        else:
            a = os.popen("sbatch --dependency="+dtype+":%s %s"%(dependency.id,self.fn)).read()
        self.id = a.split(" ")[-1].strip()
        assert self.id.isdigit(), a
        return self.id

In [3]:
#DROPOUT
j1 = Job(DIR="drop/2",FLATTEN= "SPIRAL", hidden_size=378, DSIZE=512, RUNS=0, EPOCHS=8,keep_prob=0.9)
j1.create_file("drop/2SPISING.sh")
print(j1.run())

82640


In [3]:
#SPINGLASS 420 DROPOUT
j2 = Job(DIR="drop/2BSP",FLATTEN= "SPIRAL",MODEL="SPINS1", hidden_size=378, DSIZE=512, RUNS=0, EPOCHS=30,seed=420,device='/gpu:0',keep_prob=0.9)
j2.create_file("drop/2BSPSPINS.sh")
print(j2.run())

82668


In [4]:

#j1 = Job(DIR="drop/2SP",FLATTEN= "SPIRAL",MODEL="POTTS1", DSIZE=512, RUNS=2, EPOCHS=0,keep_prob=0.9)
#j1.create_file("drop/2SPPOTTS0.sh")
#j1.run()
j3 = Job(DIR="drop/2SP",FLATTEN= "SPIRAL",MODEL="POTTS1", DSIZE=512, RUNS=0, EPOCHS=15,keep_prob=0.9)
j3.create_file("drop/2SPPOTTS1.sh")
print(j3.run())
j4 = Job(DIR="drop/2SP",FLATTEN= "SPIRAL",MODEL="POTTS1", DSIZE=512, RUNS=0, EPOCHS=15,keep_prob=0.9)
j4.create_file("drop/2SPPOTTS2.sh")
print(j4.run(j3))
j5 = Job(DIR="drop/2SP",FLATTEN= "SPIRAL",MODEL="POTTS1", DSIZE=512, RUNS=0, EPOCHS=20,keep_prob=0.9)
j5.create_file("drop/2SPPOTTS3.sh")
print(j5.run(j4))

82653
82654
82655
